In [11]:
import math
import time
import random
import copy

In [2]:
pessoas = [('Amanda', 'CWB'),
           ('Pedro', 'GIG'),
           ('Marcos', 'POA'),
           ('Priscila', 'FLN'),
           ('Jessica', 'CNF'),
           ('Paulo', 'GYN')]

destino = 'GRU'

In [3]:
voos = {}

for l in open('voos.txt'):
    origem, dest, saida, chegada, preco = l.split(',')
    voos.setdefault((origem, dest), [])
    voos[(origem, dest)].append((saida, chegada, int(preco)))

In [4]:
def print_agenda(agenda):
    for idx, voo in enumerate(agenda):
        nome = pessoas[idx][0]
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        print("%10s%10s %5s-%5s R$%3s %5s-%5s R$%3s" % (nome, origem, ida[0], ida[1], ida[2],
                                                        volta[0], volta[1], volta[2]))

agenda = [[1,3], [3, 2], [7,3], [6,3], [2,4], [5,3]]

print_agenda(agenda)

    Amanda       CWB  8:04-10:11 R$ 95 10:33-12:03 R$ 74
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 17:08-19:08 R$262 10:32-13:16 R$139
  Priscila       FLN 15:34-18:11 R$326 11:08-14:38 R$262
   Jessica       CNF  9:42-11:32 R$169 12:08-14:47 R$231
     Paulo       GYN 13:37-15:08 R$250 11:07-13:24 R$171


In [5]:
def get_minutos(hora:str):
    tempo = time.strptime(hora, '%H:%M')
    return tempo[3] * 60 + tempo[4]

get_minutos('1:10')

70

In [6]:
def custo(solucao:list):
    preco_total = 0
    # menor possível
    ultima_chegada = 0
    # maior possível
    primeira_partida = 1439 #23:59 em minutos
    total_espera = 0

    for idx, voo in enumerate(solucao):
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        preco_total += ida[2]
        preco_total += volta[2]

        # atualiza a ultima chegada ao aeroporto
        if ultima_chegada < get_minutos(ida[1]):
            ultima_chegada = get_minutos(ida[1])

        # atualiza com a primeira chegada ao aeroporto
        if primeira_partida > get_minutos(volta[0]):
            primeira_partida = get_minutos(volta[0])
        
    for idx, voo in enumerate(solucao):
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        # calcula o tanto de espera no aeroporto
        total_espera += ultima_chegada - get_minutos(ida[1])
        total_espera += get_minutos(volta[0]) - primeira_partida

    # se virou um dia, adiciona penalidade em dinheiro
    if ultima_chegada > primeira_partida:
        preco_total += 50

    return preco_total + total_espera

In [7]:
custo(agenda)

4472

## Busca randômica

In [8]:
def busca_randomica(dominio, funcao_custo, numero_solucao):
    melhor_custo = 99999999
    melhor_solucao = None

    for i in range(numero_solucao):
        solucao = [[random.randint(dominio[i][0], dominio[i][1]), random.randint(dominio[i][0], dominio[i][1])] for i in range(len(dominio))]

        custo = funcao_custo(solucao)

        if custo < melhor_custo:
            melhor_custo = custo
            melhor_solucao = solucao

    print('Melhor solução: ', melhor_solucao)
    print('Custo solução: ', melhor_custo)
    print_agenda(melhor_solucao)

    return melhor_solucao

# para cada pessoa, tem-se o numero possivel de voos.
dominio = [[0, 9]] * (len(pessoas))

busca_randomica(dominio, funcao_custo=custo, numero_solucao=1000)

Melhor solução:  [[3, 3], [0, 2], [5, 5], [4, 3], [4, 3], [4, 5]]
Custo solução:  3428
    Amanda       CWB 11:16-13:29 R$ 83 10:33-12:03 R$ 74
     Pedro       GIG  6:12-10:22 R$230  9:49-13:51 R$229
    Marcos       POA 13:40-15:38 R$137 13:37-15:33 R$142
  Priscila       FLN 12:05-15:30 R$330 11:08-14:38 R$262
   Jessica       CNF 12:44-14:17 R$134 10:33-13:11 R$132
     Paulo       GYN 12:18-14:56 R$172 14:05-15:47 R$226


[[3, 3], [0, 2], [5, 5], [4, 3], [4, 3], [4, 5]]

## Hill Climb

In [44]:
def hill_climb(domino, funcao_custo):
    # gera uma solucao randomica para iniciar o algoritmo
    solucao = busca_randomica(dominio=dominio, funcao_custo=funcao_custo,  numero_solucao=1000)

    print('Solucao aleatoria: ', solucao)

    while True:
        vizinhos = []
    
        for i in range(len(dominio)):
            # le cada valor da tupla
            for j in range(len(dominio[i])):
                 # verifica se o valor é menor que o valor maximo do dominio
                if solucao[i][j] < dominio[i][1]:
                    # faz uma copida da lista a ser modificado
                    novo_vizinho = copy.deepcopy(solucao)
                    # modifica o valor da solucao na posica [i][j]
                    novo_vizinho[i][j] = novo_vizinho[i][j] + 1
                    # adiciona a lista modificada aos vizinhos
                    vizinhos.append(novo_vizinho)
            
                # verifica se o valor é maior que o valor minimo do dominio
                if solucao[i][j] > dominio[i][0]:
                    # faz uma copida da lista a ser modificado
                    novo_vizinho = copy.deepcopy(solucao)
                    # modifica o valor da solucao na posica [i][j]
                    novo_vizinho[i][j] = novo_vizinho[i][j] - 1
                    # adiciona a lista modificada aos vizinhos
                    vizinhos.append(novo_vizinho)
        # verifica o custo da solucao aleatorea
        atual = funcao_custo(solucao)
        # adiciona o custo da funcao aleatórea a melhor
        melhor = atual

        for i in range(len(vizinhos)):
            # print(vizinhos[i])
            custo = funcao_custo(vizinhos[i])

            if custo < melhor:
                # atualiza para o melhor custo
                melhor = custo
                # registra a melhor solucao
                solucao = vizinhos[i]

        # verifica se já foram verificadas todas as solucoes
        if melhor == atual:
            break

    print('Melhor solucão: ', solucao)
    print('Melhor custo: ', melhor)
    print_agenda(agenda=solucao)
    
    return solucao

In [54]:
hill_climb(domino=dominio, funcao_custo=custo)

Melhor solução:  [[3, 1], [3, 2], [6, 3], [5, 4], [7, 3], [4, 6]]
Custo solução:  4371
    Amanda       CWB 11:16-13:29 R$ 83  8:23-10:28 R$149
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 15:23-17:25 R$232 10:32-13:16 R$139
  Priscila       FLN 14:01-17:24 R$338 12:37-15:05 R$170
   Jessica       CNF 16:43-19:00 R$246 10:33-13:11 R$132
     Paulo       GYN 12:18-14:56 R$172 15:07-17:21 R$129
Solucao aleatoria:  [[3, 1], [3, 2], [6, 3], [5, 4], [7, 3], [4, 6]]
Melhor solucão:  [[6, 3], [3, 3], [6, 3], [5, 4], [5, 3], [4, 6]]
Melhor custo:  3038
    Amanda       CWB 15:27-17:18 R$151 10:33-12:03 R$ 74
     Pedro       GIG 10:30-14:57 R$290 10:51-14:16 R$256
    Marcos       POA 15:23-17:25 R$232 10:32-13:16 R$139
  Priscila       FLN 14:01-17:24 R$338 12:37-15:05 R$170
   Jessica       CNF 14:22-16:32 R$126 10:33-13:11 R$132
     Paulo       GYN 12:18-14:56 R$172 15:07-17:21 R$129


[[6, 3], [3, 3], [6, 3], [5, 4], [5, 3], [4, 6]]